In [135]:
#!pip install requests
import requests
import os
from IPython.display import clear_output


In [125]:
BLACK = "black"
WHITE = "white"
STOCKFISH_URI = "https://stockfish.online/api/s/v2.php"

In [137]:
def clear_screen():
    # for python kernel
    if os.name == 'nt':
        _ = os.system('cls')
    else:
        _ = os.system('clear')
    clear_output(wait=True)


In [91]:
class Piece:
    def __init__(self, color, position):
        self.color = color
        self.position = position
        ## en passant, castling
        self.has_moved = False
        self.first_move = False
        self.symbol = None
        self.name = None
    def move(self, position):
        self.position = position
        # En passant check
        if self.has_moved == False:
            self.first_move = True
        else:
            self.first_move = False
        # castle, pawns 2 forward
        self.has_moved = True

    # Have this function here to avoid repetition
    def get_symbol(self):
        return self.symbol

    def get_name(self):
        return self.name

    def inbound(self, pr, pc):
        return 0 <= pr <= 7 and 0 <= pc <= 7

In [93]:
class Pawn(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.symbol = "♙" if color == BLACK else "♟"
        self.name = "p" if color == BLACK else "P"

    def get_moves(self, board, en_passant_target=None):
        move_set = []
        row, col = self.position
        # moving up or down the board
        direction = 1 if self.color == BLACK else -1
        if not board[direction + row][col]:
            if not self.has_moved:
                ## check if something is in the way on that tile
                if not board[row + direction * 2][col]:
                    move_set.append((row + direction * 2, col))
            move_set.append((direction + row, col))
        # check diagonal tiles if pawn can take
        
        if self.inbound(direction + row, col + 1):
            if board[direction + row][col + 1]:
                if board[direction + row][col + 1].color != self.color:
                    move_set.append((direction + row, col + 1))
        if self.inbound(direction + row, col - 1):
            if board[direction + row][col - 1]:
                if board[direction + row][col - 1].color != self.color:
                    move_set.append((direction + row, col - 1))
            # en passant
        if self.inbound(row, col + 1):
            if board[row][col + 1] and (row, col + 1) == en_passant_target:
                if board[row][col + 1].color != self.color:
                    move_set.append((direction + row, col + 1))
        if self.inbound(row, col - 1):
            if board[row][col - 1] and (row, col - 1) == en_passant_target:
                if board[row][col - 1].color != self.color:
                    move_set.append((direction + row, col - 1))

        ## check for promotion
        pass

        return move_set

In [95]:
class Rook(Piece):
    def __init__(self, color, position, king_side=False):
        super().__init__(color, position)
        self.symbol = "♖" if color == BLACK else "♜"
        self.name = "r" if color == BLACK else "R"
        self.king_side = king_side # true or false, use for castling rights

    def get_moves(self, board, en_passant_target=None):
        move_set = []
        # amount of change in each iteration over possible files
        directions = [(-1, 0), (1, 0), (0, 1), (0, -1)]
        for mr, mc in directions:
            # set the current_check position to the current position
            r, c = self.position
            # try for moves until out of bounds
            while self.inbound(r + mr, c + mc):
                r, c = r + mr, c + mc
                if board[r][c] is None:
                    move_set.append((r, c))
                elif board[r][c].color != self.color:
                    move_set.append((r, c))
                    break
                else:
                    break
        return move_set

    def is_king_side(self):
        return self.king_side
            

In [109]:
class King(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.symbol = "♔" if color == BLACK else "♚"
        self.name = "k" if color == BLACK else "K"
        self.q_castling_rights = True
        self.k_castling_rights = True

    def get_moves(self, board, en_passant_target=None):
        move_set = []
        # kings directions
        directions = [(-1, 0), (1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
        for mr, mc in directions:
            # set the current_check position to the current position
            r, c = self.position
            # try for moves until out of bounds
            r, c = r + mr, c + mc
            if self.inbound(r, c):
                if board[r][c] is None:
                    move_set.append((r, c))
                elif (board[r][c].color != self.color):
                    move_set.append((r, c))
        # castle king side
        # king and king side rook haven't moved
        if self.k_castling_rights:
            r, c = self.position
            tmp2 = board[r][c + 2]
            tmp = board[r][c + 1]
            if board[r][c + 1] is None and board[r][c+2] is None:
                # simulate move to the right
                board[r][c + 1] = self
                board[r][c] = None
                if not self.is_checked(board, (r, c+1)):
                    board[r][c + 2] = self
                    board[r][c + 1] = None
                    if not self.is_checked(board, (r, c+2)):
                        # ok to castle, revert action
                        move_set.append((r, c+2))
            board[r][c+2] = tmp2
            board[r][c+1] = tmp
        if self.q_castling_rights:
            r, c = self.position
            tmp = board[r][c - 1]
            tmp2 = board[r][c - 2]
            tmp3 = board[r][c - 3]
            if board[r][c - 1] is None and board[r][c-2] is None and board[r][c-3] is None:
                # simulate move to the right
                board[r][c - 1] = self
                board[r][c] = None
                if not self.is_checked(board, (r, c-1)):
                    board[r][c - 2] = self
                    board[r][c - 1] = None
                    if not self.is_checked(board, (r, c-2)):
                        # ok to castle, revert action
                        board[r][c - 3] = self
                        board[r][c - 2] = None
                        if not self.is_checked(board, (r,c-3)):
                            move_set.append((r, c-2))
            board[r][c] = self
            board[r][c-3] = tmp3
            board[r][c-2] = tmp2
            board[r][c-1] = tmp
        return move_set
    
    def is_checked(self, board, position):
        # check in each direction + knight move
        # add the position as a param so it can also check different squares when castling
        checks = 0
        # first four rook or queen, last four bishop or queen
        directions = [(-1, 0), (1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
        knight_directions = [(-1, 2), (1, 2), (2, 1), (-2, -1), (1, -2), (-1, -2), (-2, 1), (-2, -1)]
        pawn_direction = 1 if self.color == BLACK else -1
        #queen, bishop, rook
        for idx in range(len(directions)):
            # set the current_check position to the current position
            r, c = position
            mr, mc = directions[idx]
            # try for moves until out of bounds
            while self.inbound(r + mr, c + mc):
                r, c = r + mr, c + mc
                if board[r][c] is not None:
                    if board[r][c].color != self.color:
                        if idx < 4 and board[r][c].get_name().lower() in ["q", "r"]:
                            return True
                        elif idx >= 4 and board[r][c].get_name().lower() in ["q", "b"]:
                            return True
                        else:
                            break
                    else:
                        break
        #knight
        for mr, mc in knight_directions:
            # set the current_check position to the current position
            r, c = position
            # try for moves until out of bounds
            r, c = r + mr, c + mc
            if self.inbound(r, c):
                if board[r][c] is not None:
                    if (board[r][c].color != self.color):
                        if board[r][c].get_name().lower() == 'n':
                            return True
                        else:
                            # not a knight, not a threat
                            break
                    else:
                        # friendly piece in the spot
                        break
        #pawn
        r, c = position
        if self.inbound(pawn_direction + r, c + 1):
            if board[pawn_direction + r][c + 1]:
                if board[pawn_direction + r][c + 1].color != self.color:
                    if board[pawn_direction + r][c + 1].get_name().lower() == 'p':
                        return True
        if self.inbound(pawn_direction + r, c - 1):
            if board[pawn_direction + r][c - 1]:
                if board[pawn_direction + r][c - 1].color != self.color:
                    if board[pawn_direction + r][c - 1].get_name().lower() == 'p':
                        return True
        return False

    def update_castling_rights(self, king_side):
        if king_side:
            self.k_castling_rights = False
        else:
            self.q_castling_rights = False

    def get_q_castling_rights(self):
        return self.q_castling_rights

    def get_k_castling_rights(self):
        return self.k_castling_rights

In [111]:
class Queen(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.symbol = "♕" if color == BLACK else "♛"
        self.name = "q" if color == BLACK else "Q"
    def get_moves(self, board, en_passant_target=None):
        move_set = []
        # amount of change in each iteration over possible files
        directions = [(-1, 0), (1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
        
        for mr, mc in directions:
            # set the current_check position to the current position
            r, c = self.position
            # try for moves until out of bounds
            while self.inbound(r + mr, c + mc):
                r, c = r + mr, c + mc
                if board[r][c] is None:
                    move_set.append((r, c))
                elif (board[r][c].color != self.color):
                    move_set.append((r, c))
                    break
        return move_set

In [113]:
class Knight(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.symbol = "♘" if color == BLACK else "♞"
        self.name = "n" if color == BLACK else "N"
    def get_moves(self, board, en_passant_target=None):
        move_set = []
        # kings directions
        directions = [(-1, 2), (1, 2), (2, 1), (-2, -1), (1, -2), (-1, -2), (-2, 1), (2, -1)]
        for mr, mc in directions:
            r, c = self.position
            # set the current_check position to the current position
            # try for moves until out of bounds
            r, c = r + mr, c + mc
            if self.inbound(r, c):
                if board[r][c] is None:
                    move_set.append((r, c))
                elif (board[r][c].color != self.color):
                    move_set.append((r, c))
        return move_set

In [115]:
class Bishop(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.symbol = "♗" if color == BLACK else "♝"
        self.name = "b" if color == BLACK else "B"
    def get_moves(self, board, en_passant_target=None):
        move_set = []
        # amount of change in each iteration over possible files
        directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
        for mr, mc in directions:
            # set the current_check position to the current position
            r, c = self.position
            # try for moves until out of bounds
            while self.inbound(r + mr, c + mc):
                r, c = r + mr, c + mc
                if board[r][c] is None:
                    move_set.append((r, c))
                elif (board[r][c].color != self.color):
                    move_set.append((r, c))
                    break
        return move_set

In [223]:
class Chess:
    def __init__(self):
        self.columns = ["a", "b", "c", "d", "e", "f", "g", "h"]
        self.rows = [8, 7, 6, 5, 4, 3, 2, 1]
        self.board = [[None for x in range(8)] for y in range(8)]
        self.pieces = []
        self.white_king = None
        self.black_king = None
        self.checkmate = False
        self.en_passant_target = None
        self.halfmove_clock = 0# resets if pawn moves or piece takes
        self.fullmoves = 1 # + 1 when black moves
        self.stalemate = False
        self.moves = []
        self.occurred_positions = []
    def init_pieces(self):
        # 8 pawns each side
        for c in range(8):
            self.board[1][c] = Pawn(BLACK, (1, c))
            self.board[6][c] = Pawn(WHITE, (6, c))
        # init rooks
        self.board[0][0] = Rook(BLACK, (0, 0), False)
        self.board[0][7] = Rook(BLACK, (0, 7), True)
        self.board[7][0] = Rook(WHITE, (7, 0), False)
        self.board[7][7] = Rook(WHITE, (7, 7), True)
        # init knights
        self.board[0][1] = Knight(BLACK, (0, 1))
        self.board[0][6] = Knight(BLACK, (0, 6))
        self.board[7][1] = Knight(WHITE, (7, 1))
        self.board[7][6] = Knight(WHITE, (7, 6))
        # init bishops
        self.board[0][2] = Bishop(BLACK, (0, 2))
        self.board[0][5] = Bishop(BLACK, (0, 5))
        self.board[7][2] = Bishop(WHITE, (7, 2))
        self.board[7][5] = Bishop(WHITE, (7, 5))
        # init queens
        self.board[0][3] = Queen(BLACK, (0, 3))
        self.board[7][3] = Queen(WHITE, (7, 3))
        # init kings
        self.board[0][4] = King(BLACK, (0, 4))
        self.board[7][4] = King(WHITE, (7, 4))

        # pass by reference
        self.white_king = self.board[7][4]
        self.black_king = self.board[0][4]

    def is_checkmate(self):
        return self.checkmate
    
    def print_board(self):
        print("fullmoves:", self.fullmoves)
        print("halfmove clock:", self.halfmove_clock)
        col_names = "    " + "   ".join([a for a in self.columns]) + "  "
        print(col_names)
        print ('  ---------------------------------')
        for r in range(8):
            print(self.rows[r], end=" |")
            for c in range(8):
                print(f" {self.board[r][c].get_symbol()} |", end="") if self.board[r][c] else print(f" {"." if (r+c) % 2 == 0 else " "} |", end="")
            print("",self.rows[r], end="")
            print()
            print ('  ---------------------------------')
        print(col_names)

    def move(self, move, current_player):
        promoted_pieces = {
            "q": Queen,
            "n": Knight,
            "r": Rook,
            "b": Bishop
        }
        self.moves.append((move, current_player))
        start, end = None, None
        start = move[:2]
        end = move[2:4]
        promotion = move[4:]
        king = self.black_king if current_player == BLACK else self.white_king
        promote_to = None
        if self._validate_input(start) and self._validate_input(end):
            start_idx = self._input_to_index(start)
            end_idx = self._input_to_index(end)
            if isinstance(self.board[start_idx[0]][start_idx[1]], Pawn) and (end_idx[0] == 0 or end_idx[0] == 7):
                if len(promotion) == 1:
                    if promotion[0].lower() in ['q', 'r', 'n', 'b']:
                        if end_idx[0] == 0 or end_idx[0] == 7:
                            promote_to = promoted_pieces[promotion[0].lower()](self.board[start_idx[0]][start_idx[1]].color, (start_idx[0], start_idx[1]))
                            return self._update_board(start_idx, end_idx, current_player, promote_to)
                return False # should promote, but wont allow as missing to what piece it should promote to
            return self._update_board(start_idx, end_idx, current_player, promote_to)
        return False

    def _get_valid_moves(self, player):
        valid_moves = []
        for x in range(8):
            for y in range(8):
                if self.board[x][y]:
                    piece = self.board[x][y]
                    moves = piece.get_moves(self.board, self.en_passant_target)
                    if piece.color == player:
                        for m in moves:
                            r, c = piece.position
                            dr, dc = m
                            tmp = self.board[dr][dc]
                            self.board[dr][dc] = self.board[r][c]
                            self.board[r][c] = None
                            king = self.black_king if player == BLACK else self.white_king
                            king_pos = king.position
                            if isinstance(self.board[dr][dc], King):
                                king_pos = (dr, dc)
                            if not king.is_checked(self.board, king_pos):
                                valid_moves.append(m)
                            self.board[r][c] = self.board[dr][dc]
                            self.board[dr][dc] = tmp
        return valid_moves
    
    def _validate_input(self, string):
        first, second = None, None
        # castling later
        try:
            str_arr = list(string)
            if len(str_arr) == 2:
                if str_arr[0] in self.columns and int(str_arr[1]) in self.rows:
                    return True
            raise Exception
        except Exception as e:
            print(f"Failed to parse input, please retry.")
            return False
    
    def _input_to_index(self, move):
        str_arr = list(move)
        col_move, row_move = str_arr
        # ord("a") = 97, ord("b") = 98, ... so ord(col) - ord('a') = index
        col_move_idx = ord(col_move) - ord("a")
        row_move_idx = 8 - int(row_move)
        return (row_move_idx, col_move_idx)

    def _update_board(self, move_origin, new_move, current_player, promote_to=None):
        reset_half_move = False
        if self.board[move_origin[0]][move_origin[1]]:
            piece = self.board[move_origin[0]][move_origin[1]]
            if piece.color == current_player:
                if new_move in piece.get_moves(self.board, self.en_passant_target):
                    # cannot swap directly as there might be a piece kill
                    # en passant later
                    king = self.black_king if current_player == BLACK else self.white_king
                    if isinstance(piece, King) and new_move[1] - move_origin[1] in [2, -2]:
                        # castling, king moves twice
                        king_side = False if new_move[1] - move_origin[1] == -2 else True
                        # rook must be in original place as it was checked in get_moves
                        rook = self.board[move_origin[0]][7 if king_side else 0]
                        self.board[move_origin[0]][5 if king_side else 3] = rook
                        rook.move((move_origin[0], 5 if king_side else 3))
                        self.board[move_origin[0]][7 if king_side else 0] = None
                        king.update_castling_rights(True)
                        king.update_castling_rights(False)
                    if isinstance(piece, Pawn) and self.board[new_move[0]][new_move[1]] == None and new_move[1] != move_origin[1]:
                        if isinstance(piece, Pawn) and move_origin[1] + 1 == self.en_passant_target[1]:
                            self.board[move_origin[0]][move_origin[1]] = None
                            tmp = self.board[move_origin[0]][move_origin[1] + 1]
                            self.board[new_move[0]][new_move[1]] = piece
                            self.board[move_origin[0]][move_origin[1] + 1] = None
                            piece.move(new_move)
                        elif isinstance(piece, Pawn) and move_origin[1] - 1 == self.en_passant_target[1]:
                            self.board[move_origin[0]][move_origin[1]] = None
                            tmp = self.board[move_origin[0]][move_origin[1] - 1]
                            self.board[new_move[0]][new_move[1]] = piece
                            self.board[move_origin[0]][move_origin[1] - 1] = None
                            piece.move(new_move)
                    else:
                        self.board[move_origin[0]][move_origin[1]] = None
                        tmp = self.board[new_move[0]][new_move[1]]
                        if tmp:
                            reset_half_move = True
                        self.board[new_move[0]][new_move[1]] = piece
                        piece.move(new_move)
                    # if check after the move, revert it
                    if king.is_checked(self.board, king.position):
                        self.board[move_origin[0]][move_origin[1]] = piece
                        piece.move(move_origin)
                        self.board[new_move[0]][new_move[1]] = tmp
                        return False
                    if isinstance(piece, King):
                        king.update_castling_rights(False)
                        king.update_castling_rights(True)
                    if isinstance(piece, Rook):
                        king.update_castling_rights(piece.is_king_side)
                    # check if checkmate for the other side, switch king
                    other_player = BLACK if current_player == WHITE else WHITE
                    other_king = self.black_king if other_player == BLACK else self.white_king
                    if other_king.is_checked(self.board, other_king.position):
                        print("Check")
                        op = self._get_valid_moves(other_player)
                        if len(op) == 0:
                            self.checkmate = True
                    elif len(self._get_valid_moves(other_player)) == 0:
                        self.stalemate = True
                    if isinstance(piece, Pawn) and new_move[0] - move_origin[0] in [2, -2]:
                        self.en_passant_target = new_move
                    else:
                        self.en_passant_target = None
                    if promote_to != None:
                        self.board[new_move[0]][new_move[1]] = promote_to
                    if current_player == BLACK:
                        self.fullmoves += 1
                    if isinstance(piece, Pawn) or reset_half_move:
                        self.halfmove_clock = 0
                    if self.halfmove_clock == 50:
                        self.stalemate = True
                    if self.check_three_move_rule():
                        self.stalemate = True
                    else:
                        self.halfmove_clock += 1
                    return True
        return False


    def check_three_move_rule(self):
        piece_fen = self.get_fen(None, True)
        count = 0
        for x in self.occurred_positions:
            if piece_fen == x:
                count += 1
        self.occurred_positions.append(piece_fen)
        return count >= 3
    
    def get_fen(self, current_player, position_only=False):
        fen_pieces = ""
        for row in self.board:
            empty = 0
            for cell in row:
                if cell:
                    if empty > 0:
                        fen_pieces += str(empty)
                        empty = 0
                    fen_pieces += cell.name
                else:
                    empty += 1
            if empty > 0:
                fen_pieces += str(empty)
            fen_pieces += '/'
        # remove / at the end
        fen_pieces = fen_pieces.rstrip('/')
        active_color = "w" if current_player == WHITE else "b"
        castling = ""
        if self.white_king.get_k_castling_rights():
            castling += 'K'
        if self.white_king.get_q_castling_rights():
            castling += 'Q'
        if self.black_king.get_k_castling_rights():
            castling += 'k'
        if self.black_king.get_q_castling_rights():
            castling += 'q'
        castling = castling if castling else '-'
        if not self.en_passant_target:
            en_passant = '-'
        else:
            en_passant = f"{self.columns[self.en_passant_target[0]]}{self.rows[self.en_passant_target[1]]}"
        halfmove = str(self.halfmove_clock)
        fullmove = str(self.fullmoves)
        if position_only:
            return fen_pieces
        fen = f"{fen_pieces} {active_color} {castling} {en_passant} {halfmove} {fullmove}"
        return fen

    def is_stalemate(self):
        return self.stalemate
        

In [231]:
chess = Chess()
chess.init_pieces()
current_player = [WHITE, BLACK]
player1 = input("White - player (p) or stockfish (s): ")
player2 = input("Black - player (p) or stockfish (s): ")
p1_depth = 0
p2_depth = 0
if player1 == 's':
    p1_depth = int(input("White - stockfish depth (0-15): "))
if player2 == 's':
    p2_depth = int(input("White - stockfish depth (0-15): "))
players = [player1, player2]
player_depths = [p1_depth, p2_depth]
def scenario1():
    # castling + moves
    chess.move("e2e4", WHITE)
    chess.move("e7e5", BLACK)
    chess.move("d1h5", WHITE)
    chess.move("b8c6", BLACK)
    chess.move("f1c4", WHITE)
    chess.move("a7a6", BLACK)
    chess.move("b1c3", WHITE)
    chess.move("d2d3", WHITE)
    chess.move("c1d2", WHITE)
    chess.move("d8h4", BLACK)
    chess.move("a2a3", WHITE)
    chess.move("h4g4", BLACK)
    chess.move("e1g1", WHITE) # shouldnt work, square in check
    chess.move("b7b6", BLACK)
    chess.move("g1h3", WHITE)
    chess.move("a6a5", BLACK)
    chess.move("e1g1", WHITE)
    chess.move("b6b5", BLACK)
def scenario2():
    chess.move("e2e4", WHITE)
    chess.move("e7e5", BLACK)
    chess.move("d1h5", WHITE)
    chess.move("b8c6", BLACK)
    chess.move("f1c4", WHITE)
    chess.move("a7a6", BLACK)
    chess.move("b1c3", WHITE)
    chess.move("d2d3", WHITE)
    chess.move("c1d2", WHITE)
    chess.move("d8h4", BLACK)
    chess.move("h5f7", WHITE) #check
    chess.move("e8d8", BLACK) #out of check
    chess.move("f7f8", WHITE) #checkmate
def scenario3():
    chess.move("e2e4", WHITE)
    chess.move("e4e5", WHITE)
    chess.move("d7d5", BLACK)
    chess.move("e5d6", WHITE)

def scenario4():
    chess.move("e2e4", WHITE)
    chess.move("e4e5", WHITE)
    chess.move("d7d5", BLACK)
    chess.move("e5d6", WHITE)
    chess.move("d6c7", WHITE)
    chess.move("c7b8", WHITE)
def scenario5():
    chess.move("c7c5", BLACK)
    chess.move("b8c6", BLACK)
    chess.move("d7d5", BLACK)
    chess.move("d8c7", BLACK)
    chess.move("c8d7", BLACK)
    chess.move("e8c8", BLACK)
    chess.print_board()

def custom_scenario():
    x = [('e2e4', 'white'), ('e7e5', 'black'), ('d2d4', 'white'), ('e5d4', 'black'), ('d1d4', 'white'), ('g8f6', 'black'), ('f1c4', 'white'), ('b8c6', 'black'), ('d4e3', 'white'), ('f6g4', 'black'), ('e3f4', 'white'), ('f8b4', 'black'), ('c2c3', 'white'), ('d7d5', 'black'), ('c4d5', 'white'), ('c6e5', 'black'), ('d5f7', 'white'), ('e5f7', 'black'), ('c3b4', 'white'), ('f7e5', 'black'), ('f4d2', 'white'), ('e5d3', 'black'), ('e1f1', 'white'), ('g4f2', 'black'), ('g1f3', 'white'), ('f2h1', 'black'), ('b1c3', 'white'), ('e8g8', 'black'), ('f1g1', 'white'), ('f8f3', 'black'), ('g2f3', 'white'), ('h1f2', 'black'), ('c3d5', 'white'), ('d8d6', 'black'), ('d5f4', 'white'), ('d6f4', 'black'), ('d2f2', 'white'), ('d3c1', 'black'), ('a2a3', 'white'), ('c8e6', 'black'), ('g1g2', 'white'), ('a8d8', 'black'), ('a1b1', 'white'), ('d8d2', 'black'), ('f2d2', 'white'), ('f4d2', 'black'), ('g2h1', 'white'), ('e6h3', 'black'), ('b2b3', 'white'), ('d2d1', 'black')]
    for t in x:
        chess.move(t[0], t[1])
        chess.print_board()
# scenario1()
chess.print_board()
def mainloop():
    #try: 
    turn = 0
    while not chess.is_checkmate() and not chess.is_stalemate():
        move = ""
        fen = chess.get_fen(current_player[turn % 2])

        if players[turn % 2] == 's':
            depth = player_depths[turn % 2]
            res = requests.get(STOCKFISH_URI, params={"fen": fen, "depth": depth})
            if res.status_code == 200:
                res_json = res.json()
                # Extract the 'bestmove' value
                bestmove = res_json.get("bestmove", "")
                move = bestmove.split()[1]
            else:
                raise Exception
        else:
            move = input("Enter move (e.g. e2e4): ")
            if move== "q":
                break
        success = chess.move(move, current_player[turn % 2])
        if success:
            turn += 1
            clear_screen()
            print(fen)
            chess.print_board()
    if chess.is_checkmate():
        print(f"Checkmate - {current_player[turn % 2]} wins")
    if chess.is_stalemate():
        print("Stalemate!")
    #except Exception as e:
    #    print(e)
#chess.move("g8f6", BLACK)
#custom_scenario()
mainloop()
#scenario5()
#chess.print_board()

8/1p3kp1/4p3/1B1pP1p1/p2b2P1/P5K1/2R2P2/8 w - - 18 47
fullmoves: 47
halfmove clock: 18
    a   b   c   d   e   f   g   h  
  ---------------------------------
8 | . |   | . |   | . |   | . |   | 8
  ---------------------------------
7 |   | ♙ |   | . |   | ♔ | ♙ | . | 7
  ---------------------------------
6 | . |   | . |   | ♙ |   | . |   | 6
  ---------------------------------
5 |   | ♝ |   | ♙ | ♟ | . | ♙ | . | 5
  ---------------------------------
4 | ♙ |   | . | ♗ | . |   | ♟ |   | 4
  ---------------------------------
3 | ♟ | . |   | . |   | . | ♚ | . | 3
  ---------------------------------
2 | . |   | . |   | ♜ | ♟ | . |   | 2
  ---------------------------------
1 |   | . |   | . |   | . |   | . | 1
  ---------------------------------
    a   b   c   d   e   f   g   h  
Stalemate!
